<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-422-salleh/Exercises/day-14/Audit_Governance/AI_Audit_Governance_Practice_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem Statement** – AI Governance, Bias Detection \& Auditing in Income Prediction Model

You are part of an **AI governance team** at a tech company developing a predictive analytics system to identify individuals likely to earn more than **\$50K/year**.

The model must comply with **AI governance principles**, focusing on **fairness, transparency, and accountability**.

Your tasks in this exercise are to:

- **Detect and measure bias** in predictions across demographic groups (here we will use `"sex"` as a sensitive attribute: Male vs Female).
- **Provide model transparency** using **SHAP explainability** to understand the most influential features driving predictions.
- **Implement AI auditing** by logging performance metrics, bias statistics, and interpretability results.
- **Generate a compliance report** that can be reviewed by internal governance boards or external regulators.

**Business Context:**
Such a model must not unfairly disadvantage certain protected groups. This auditing ensures the system adheres to ethical AI frameworks (like IEEE, OECD, NIST AI RMF) and privacy regulations.



In [1]:
# Install necessary packages
!pip install shap scikit-learn pandas -q

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap
import datetime

In [2]:
# Dataset Collection Code*
from sklearn.datasets import fetch_openml

# Load Adult Census dataset from OpenML
adult_data = fetch_openml(name='adult', version=2, as_frame=True)

# Extract features and target
X = adult_data.data
y = adult_data.target

# Convert target to binary: >50K = 1, <=50K = 0
y = y.map({'>50K': 1, '<=50K': 0})

print("Dataset shape:", X.shape)
print("Target distribution:\n", y.value_counts())

Dataset shape: (48842, 14)
Target distribution:
 class
0    37155
1    11687
Name: count, dtype: int64


In [3]:
# Encode categorical variables (One-Hot Encoding)
X = pd.get_dummies(X, drop_first=True)

# Sensitive attribute: We reconstruct 'sex' column for bias detection
# Original dataset contains 'sex', we need it separately for bias audit
adult_original = adult_data.data
sensitive_attr = adult_original['sex'].map({'Male': 1, 'Female': 0})  # 1=Male, 0=Female

X['sex_attr'] = sensitive_attr  # Keep a separate column


In [4]:
# -------------------
# Train-Test Split
# -------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [5]:
# -------------------
# Model Training
# -------------------
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

In [6]:
# -------------------
# Model Evaluation
# -------------------
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# -------------------
# AI Auditing Step 1: Log metrics
# -------------------
audit_log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "classification_report": classification_report(y_test, y_pred, output_dict=True),
    "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
}
print("\nAudit Log Initial Entry:\n", audit_log)



Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91     11233
           1       0.73      0.63      0.68      3420

    accuracy                           0.86     14653
   macro avg       0.81      0.78      0.79     14653
weighted avg       0.85      0.86      0.85     14653


Audit Log Initial Entry:
 {'timestamp': '2025-08-14T09:02:15.108300', 'classification_report': {'0': {'precision': 0.8918271287976037, 'recall': 0.9277129885159797, 'f1-score': 0.9094161794222881, 'support': 11233.0}, '1': {'precision': 0.7264150943396226, 'recall': 0.6304093567251462, 'f1-score': 0.6750156543519098, 'support': 3420.0}, 'accuracy': 0.8583225278100047, 'macro avg': {'precision': 0.8091211115686132, 'recall': 0.779061172620563, 'f1-score': 0.792215916887099, 'support': 14653.0}, 'weighted avg': {'precision': 0.8532200750989554, 'recall': 0.8583225278100047, 'f1-score': 0.8547072600378144, 'support': 14653.0}}, 'confusion_mat

In [8]:
# -------------------
# Bias Detection by Sex
# -------------------
# Retrieve sensitive attribute from test set
test_results = X_test.copy()
test_results['actual'] = y_test.values
test_results['predicted'] = y_pred

grouped = test_results.groupby('sex_attr').agg(
    total=('actual', 'count'),
    positive_rate_actual=('actual', 'mean'),
    positive_rate_predicted=('predicted', 'mean')
)

grouped.index = grouped.index.map({0: 'Female', 1: 'Male'})
print("\nOutcome rates by Sex:\n", grouped)

# Add bias report to audit log
bias_report = grouped.to_string()
audit_log['bias_report'] = bias_report


/tmp/ipython-input-3195895873.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = test_results.groupby('sex_attr').agg(


TypeError: category dtype does not support aggregation 'mean'

In [ ]:
# -------------------
# Model Transparency with SHAP
# -------------------
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test)

# Plot summary for class '1' (>50K income)
shap.summary_plot(shap_values[1], X_test, show=True)


In [ ]:
# -------------------
# Compliance & Audit Documentation
# -------------------
compliance_notes = f"""
Model Compliance and Audit Report:
-----------------------------------
Timestamp: {audit_log['timestamp']}

Bias Detection:
Outcome disparities between sex groups:
{bias_report}

Model Performance:
- Detailed metrics in classification report
- Confusion matrix: {audit_log["confusion_matrix"]}

Transparency:
- SHAP used to interpret feature importance and influence
- Explainability provided for decision accountability

Governance & Auditing:
- Metrics, bias analysis, and interpretability documented
- Ready for review under AI governance frameworks and regulatory requirements
"""

print(compliance_notes)